<html>
<div>
  <img src="https://www.engineersgarage.com/wp-content/uploads/2021/11/TCH36-01-scaled.jpg" width=360px width=auto style="vertical-align: middle;">
  <span style="font-family: Georgia; font-size:30px; color: white;"> <br/> University of Tehran <br/> AI_CA2 <br/> Spring 02 </span>
</div>
<span style="font-family: Georgia; font-size:15pt; color: white; vertical-align: middle;"> low_mist - std id: 810100186 </span>
</html>

in this notebook we are to learn about adversarial search and minimax method to solve them.

## Problem Description
in this problem we have to play [othello](https://www.eothello.com) against computer which only does random moves. we have to find a way to defeat it and we will also look into some optimization like alpha-beta pruning in order to improve the running time of the algorithm. 

In [ ]:
from __future__ import annotations
import random
import time
import turtle
import math
import time
from copy import deepcopy
from enum import Enum

### Othello UI
It will draw our board for us to have better experience using the app.

In [ ]:
class OthelloUI:
    def __init__(self, board_size = 6, square_size = 60):
        self.board_size = board_size
        self.square_size = square_size
        self.screen = turtle.Screen()
        self.screen.setup(self.board_size * self.square_size + 50, self.board_size * self.square_size + 50)
        self.screen.bgcolor('white')
        self.screen.title('Othello low mist')
        self.pen = turtle.Turtle()
        self.pen.hideturtle()
        self.pen.speed(0)
        turtle.tracer(0, 0)

    def draw_board(self, board):
        self.pen.penup()
        x, y = -self.board_size / 2 * self.square_size, self.board_size / 2 * self.square_size
        for i in range(self.board_size):
            self.pen.penup()
            for j in range(self.board_size):
                self.pen.goto(x + j * self.square_size, y - i * self.square_size)
                self.pen.pendown()
                self.pen.fillcolor('green')
                self.pen.begin_fill()
                self.pen.setheading(0)
                for _ in range(4):
                    self.pen.forward(self.square_size)
                    self.pen.right(90)
                self.pen.penup()
                self.pen.end_fill()
                self.pen.goto(x + j * self.square_size + self.square_size / 2,
                              y - i * self.square_size - self.square_size + 5)
                if board[i][j] == 1:
                    self.pen.fillcolor('white')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()
                elif board[i][j] == -1:
                    self.pen.fillcolor('black')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()

        turtle.update()

## The Main Game Class
The main class `Othello` is defined here which has different methods:
- `constructor`: we initialize some useful values that will be used later.
Some utilities for determining winner, getting valid moves, check for terminal and so forth.
- `get_winner`
- `get_valid_moves`
- `terminal_test`
- `get_cpu_move`
- `get_human_move`  
And the main method is play which will run and gives us the result. 1 means that we have won, 0 indicates draw and -1 shows that we lost.  
Since we want to run algorithm in different modes (i.e. with and without pruning) with different minimax depth and check their running times we put these two setter to alter some values.
- `set_minimax_depth`
- `set_pruning`  
And there are some functions for evaluation such as
- `count_corners`
- `count_borders`
- ...  
And then we get a weighted mean based on coefficients defined in constructor

In [ ]:
HUMAN, COMPUTER = 1, -1

Move = tuple[int, int]

class Othello:
    def __init__(self, ui, minimax_depth = 1, prune = True):
        self.size = 6
        self.ui = OthelloUI(self.size) if ui else None
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])
        self.minimax_depth = minimax_depth
        self.prune = prune
        self.CORNER_WEIGHT = 3
        self.BORDER_WEIGHT = 2
        self.TOTAL_WEIGHT = 1
        
    def set_minimax_depth(self, depth: int):
        self.minimax_depth = depth
        
    def set_pruning(self, prune: bool):
        self.prune = prune

    def get_winner(self):
        white_count = sum([row.count(HUMAN) for row in self.board])
        black_count = sum([row.count(COMPUTER) for row in self.board])
        if white_count > black_count:
            return HUMAN
        elif white_count < black_count:
            return COMPUTER
        else:
            return 0

    def get_valid_moves(self, player):
        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)

    def make_move(self, player, move):
        i, j = move
        self.board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        self.board[cx][cy] = player

    def get_cpu_move(self):
        moves = self.get_valid_moves(COMPUTER)
        if len(moves) == 0:
            return None
        return random.choice(moves)

    def get_human_move(self):
        value, move = self.minimax(self.minimax_depth, HUMAN)
        return move
    
    def minimax(self, depth: int, turn: int, alpha: float = -math.inf, beta: float = math.inf) -> tuple[int, Move]:
                # moves = self.get_valid_moves(HUMAN)
                # if len(moves) == 0:
                #     return None, None
                # return None, random.choice(moves)
        if self.terminal_test():
            return self.heuristic(), None
        
        if depth <= 0:
            return self.heuristic(), None
        
        backup_board = deepcopy(self.board)
        optimal_move = None
        
        if turn == HUMAN:
            node_value = -math.inf
            for move in self.get_valid_moves(HUMAN):
                self.make_move(HUMAN, move)
                value, move = self.minimax(depth - 1, -turn, alpha, beta)
                self.board = backup_board
                if value > node_value:
                    optimal_move = move
                    node_value = value
                    if self.prune and node_value >= beta:
                        break
                    alpha = max(alpha, value)
                
            return node_value, optimal_move 
        
        elif turn == COMPUTER:
            node_value = math.inf
            for move in self.get_valid_moves(HUMAN):
                self.make_move(COMPUTER, move)
                value, move = self.minimax(depth - 1, -turn, alpha, beta)
                self.board = backup_board
                if value < node_value:
                    optimal_move = move
                    node_value = value
                    if self.prune and node_value <= alpha:
                        break
                    beta = min(beta, value)
                    
            return node_value, optimal_move 
                

    def evaluate(self,  move: Move) -> int:
        board = deepcopy(self.board)
        self.make_move(move)
        value = self.heuristic()
        self.board = board
        return value    # bullshit
        
    def heuristic(self) -> int:
        return self.CORNER_WEIGHT * (self.count_corners(HUMAN) - self.count_corners(COMPUTER)) + \
               self.BORDER_WEIGHT * (self.count_borders(HUMAN) - self.count_borders(COMPUTER)) + \
               self.TOTAL_WEIGHT * (self.count_total(HUMAN) - self.count_total(COMPUTER))    
    
    def count_corners(self, player: int) -> int:
        return self.board[0][0] == player + self.board[0][-1] == player + \
               self.board[-1][0] == player + self.board[-1][-1] == player
               
    def count_borders(self, player: int) -> int:
        sum = 0
        for i in range(self.size):
            sum += self.board[0][i] == player
            sum += self.board[-1][i] == player
            sum += self.board[i][0] == player
            sum += self.board[i][-1] == player
        return sum
    
    def count_total(self, player: int) -> int:
        return sum([row.count(player) for row in self.board])
        
    def terminal_test(self):
        return len(self.get_valid_moves(HUMAN)) == 0 and len(self.get_valid_moves(COMPUTER)) == 0

    def play(self):
        winner = None
        while not self.terminal_test():
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == HUMAN:
                move = self.get_human_move()
                if move:
                    self.make_move(self.current_turn, move)
            else:
                move = self.get_cpu_move()
                if move:
                    self.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
            if self.ui:
                self.ui.draw_board(self.board)
                time.sleep(1)
        winner = self.get_winner()
        return winner
    
othello = Othello(True)
othello.play()